### ABL Scan

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import yfinance as yf
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import ta

import warnings
warnings.filterwarnings('ignore')

# import date library
from datetime import date
today = date.today().strftime("%m_%d_%y")

print('pandas v.',pd.__version__,
      '\numpy v.',np.__version__,
      '\nyfinace v.',yf.__version__)

pandas v. 1.5.2 
umpy v. 1.23.5 
yfinace v. 0.2.12


In [2]:
df_symbol = pd.read_csv('symbol_SET.csv')
tickers = df_symbol[df_symbol['Symbol'].str.contains('-R.BK') == False].reset_index(drop=True)
tickers.head()

,Symbol,Name
0,24CS.BK,Twenty - Four Con & Supply Public Company Limited
1,2S.BK,2S Metal Public Company Limited
2,3K-BAT.BK,Thai Energy Storage Technology Public Company ...
3,7UP.BK,Seven Utilities and Power Public Company Limited
4,A.BK,Areeya Property Public Company Limited


In [3]:
def download_history(ticker = 'ADVANC.BK',period='1y',interval='1d'):
    """
    ticker: equity symbol in yahoo finance
    periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max (time period)
    interval: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo (trading interval)
    fetch data by interval (including intraday if period < 60 days)
    """
    data = yf.download(tickers = ticker, period = period, interval = interval)
    data['Symbol'] = ticker

    return data

In [4]:
def abl_trend_following(ticker = 'ADVANC.BK',period='1y',interval='1d'):
    """
    ticker: equity symbol in yahoo finance
    periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max (time period)
    interval: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo (trading interval)
    fetch data by interval (including intraday if period < 60 days)
    """
    data = yf.download(tickers=ticker,period=period,interval=interval)
    o = data['Open']
    h = data['High']
    l = data['Low']
    c = data['Close']
    
    data['SMA20'] = ta.trend.SMAIndicator(c,20).sma_indicator()
    data['SMA50'] = ta.trend.SMAIndicator(c,50).sma_indicator()
    data['SMA200'] = ta.trend.SMAIndicator(c,200).sma_indicator()
    
    DCC = ta.volatility.DonchianChannel(h,l,c,window=20,offset=0)
    data['HHV'] = DCC.donchian_channel_hband()
    data['LLV'] = DCC.donchian_channel_lband()
    
    ADX = ta.trend.ADXIndicator(h,l,c,window=14)
    data['ADX'] = ADX.adx()
    
    MACD = ta.trend.MACD(c,window_slow=26,window_fast=12,window_sign=9)
    data['MACD'] = MACD.macd()
    data['MACD_Sig'] = MACD.macd_signal()
    data['MACD_HHV'] = ta.volatility.donchian_channel_hband(data['MACD'],data['MACD'],data['MACD'],window=50,offset=0)
    data['MACD_LLV'] = ta.volatility.donchian_channel_lband(data['MACD'],data['MACD'],data['MACD'],window=50,offset=0)
    
    
    BB = ta.volatility.BollingerBands(c,window=20,window_dev=2)
    data['BB_H'] = BB.bollinger_hband()
    data['BB_H_Sig'] = BB.bollinger_hband_indicator()
    data['BB_L'] = BB.bollinger_lband()
    data['BB_L_Sig'] = BB.bollinger_lband_indicator()
    
    data['RSI'] = ta.momentum.RSIIndicator(c,window=14).rsi()
    
    STO = ta.momentum.StochasticOscillator(h,l,c,window=14,smooth_window=3)
    data['STO'] = STO.stoch()
    data['STO_Sig'] = STO.stoch_signal()
    
    data['SMA20>SMA50'] = np.where(data['SMA20']>data['SMA50'],True,False)
    data['SMA50>SMA200'] = np.where(data['SMA50']>data['SMA200'],True,False)
    data['SMA20_cross_SMA50'] = np.where((data['SMA20']>data['SMA50'])&(data['SMA20'].shift(1)<=data['SMA50'].shift(1)),True,False)
    data['SMA50_cross_SMA200'] = np.where((data['SMA50']>data['SMA200'])&(data['SMA50'].shift(1)<=data['SMA200'].shift(1)),True,False)
    data['HHV_Sig'] = np.where(data['HHV']>data['HHV'].shift(1),True,False)
    data['LLV_Sig'] = np.where(data['LLV']<data['LLV'].shift(1),True,False)
    data['ADX_Sig'] = np.where(data['ADX']>=30,True,False)
    data['Sig_MACD'] = np.where(data['MACD']>0,True,False)
    data['Sig_cross_MACD'] = np.where((data['MACD_Sig']>data['MACD'])&(data['MACD_Sig'].shift(1)<=data['MACD'].shift(1))&(data['MACD']>0),True,False)
    
    return data.tail(1)

In [5]:
def plot_candlestick(df):
    date = df.index
    o = df['Open']
    h = df['High']
    l = df['Low']
    c = df['Close']
    
    fig = make_subplots(rows=3,cols=1,shared_xaxes=True)
    fig.add_trace(go.Candlestick(x=date,open=o,high=h,low=l,close=c),row=1,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['SMA20'],mode='lines',line = dict(color='green'), name='SMA20'),row=1,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['SMA50'],mode='lines',line = dict(color='goldenrod'), name='SMA50'),row=1,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['SMA200'],mode='lines',line = dict(color='red',width=3), name='SMA200'),row=1,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['HHV'],mode='lines',line = dict(color='blue'), name='HHV'),row=1,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['LLV'],mode='lines',line = dict(color='brown'), name='LLV'),row=1,col=1)
    
    fig.add_trace(go.Scatter(x=date, y=df['RSI'],mode='lines',line = dict(color='blue'),name='RSI'),row=2,col=1)
    fig.add_trace(go.Scatter(x=date, y=df['ADX'],mode='lines',line = dict(color='blue'),name='ADX'),row=3,col=1)
#     fig.add_trace(go.Scatter(x=date,y=30),row=3,col=1)
    
    fig.update_layout(xaxis_rangeslider_visible=False)
    
    fig.update_layout(height=1000, width=1200, title_text='Candelstick and indicator')
    fig.show()

In [6]:
# valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
period = '2y'

# fetch data by interval including intraday if period < 60 days)
# valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
interval = '1d'

df = pd.DataFrame()
for ticker in tqdm(tickers['Symbol'][250:]):
    print(ticker)
    try:
        data = abl_trend_following(ticker,period,interval)
        data['Ticker'] = ticker
        df = pd.concat([df,data])
    except:
        print('fail')
    # plot_candlestick(data)
    # print(data.tail(5))

print(df)
df.to_excel('signal.xlsx')

  0%|          | 0/626 [00:00<?, ?it/s]

FSMART.BK
[*********************100%***********************]  1 of 1 completed

  0%|          | 1/626 [00:00<06:49,  1.53it/s]


FSS.BK
[*********************100%***********************]  1 of 1 completed

  0%|          | 2/626 [00:00<04:13,  2.46it/s]

...


ZEN.BK
[*********************100%***********************]  1 of 1 completed

100%|█████████▉| 625/626 [02:28<00:00,  4.40it/s]


ZIGA.BK
[*********************100%***********************]  1 of 1 completed

100%|██████████| 626/626 [02:28<00:00,  4.20it/s]


             Open   High    Low  Close  Adj Close    Volume    SMA20    SMA50  \
Date                                                                            
2023-06-02  10.10  10.10   9.85   9.95       9.95   2208448   9.6000   9.8370   
2023-06-02   2.88   2.92   2.86   2.86       2.86    225614   3.0040   3.3036   
2023-06-02   1.68   1.71   1.68   1.70       1.70     67951   1.7035   1.6670   
2023-06-02   2.28   2.28   2.20   2.26       2.26    615800   2.1430   2.1056   
2023-06-02  11.00  11.10  10.90  11.00      11.00    492519  11.0850  11.0960   
...           ...    ...    ...    ...        ...       ...      ...      ...   
2023-06-02   8.25   8.25   8.05   8.05       8.05   1173303   7.5725   7.4050   
2023-06-02   2.70   2.78   2.68   2.74       2.74  11070030   2.6860   2.8500   
2023-06-02  13.30  13.30  13.30  13.30      13.30      1910  13.2100  13.5720   
2023-06-02  12.20  12.40  12.20  12.40      12.40   1124200  13.3700  15.4260   
2023-06-02   2.06   2.08   

In [7]:
df

,Open,High,Low,Close,Adj Close,Volume,SMA20,SMA50,SMA200,HHV,...,SMA_Sig1,SMA_Sig2,SMA20_cross_SMA50,SMA50_cross_SMA200,HHV_Sig,LLV_Sig,ADX_Sig,Sig_MACD,Sig_cross_MACD,Ticker
Date,,,,,,,,,,,,,,,,,,,,,
2023-06-02,10.10,10.10,9.85,9.95,9.95,2208448,9.6000,9.8370,14.98875,10.40,...,0,0,0,0,0,0,0,1,0,FSMART.BK
2023-06-02,2.88,2.92,2.86,2.86,2.86,225614,3.0040,3.3036,3.56960,3.38,...,0,0,0,0,0,0,0,0,0,FSS.BK
2023-06-02,1.68,1.71,1.68,1.70,1.70,67951,1.7035,1.6670,1.52660,1.85,...,1,1,0,0,0,0,0,1,0,FTE.BK
2023-06-02,2.28,2.28,2.20,2.26,2.26,615800,2.1430,2.1056,2.18370,2.40,...,1,0,0,0,0,0,1,1,0,FTI.BK
2023-06-02,11.00,11.10,10.90,11.00,11.00,492519,11.0850,11.0960,10.78450,11.30,...,0,1,0,0,0,0,0,0,0,FTREIT.BK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-02,8.25,8.25,8.05,8.05,8.05,1173303,7.5725,7.4050,8.43625,8.40,...,1,0,0,0,0,0,0,1,0,YGG.BK
2023-06-02,2.70,2.78,2.68,2.74,2.74,11070030,2.6860,2.8500,2.35430,3.00,...,0,1,0,0,0,0,0,0,0,YONG.BK
2023-06-02,13.30,13.30,13.30,13.30,13.30,1910,13.2100,13.5720,14.17600,14.00,...,0,0,0,0,0,0,0,0,0,YUASA.BK
